In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_html(url) :
    html = ""
    resp = requests.get(url)

    if resp.status_code == 200 :
        html = resp.text
    return html
html = get_html('http://www.boonyang24.com/?sub=boonyang_info/info_list&do1=&si1=&boonryu01=&boonryu02=&page=2&keyword=&tp=&cate=&area_code=&work_year=&hyun_cate=')
# print(html)

c = get_html('http://www.boonyang24.com/?sub=boonyang_info/info_list&do1=&si1=&boonryu01=&boonryu02=&page=2&keyword=&tp=&cate=&area_code=&work_year=&hyun_cate=')
soup = BeautifulSoup(c, 'html.parser')

boonyang = soup.find("div",{"class":"list_type_boonyang"})
#----------------------------------------------------------------------
ul_li = boonyang.find("ul", {"class":"top"})
ul_li.text
print(ul_li)
lis = ul_li.findAll("li") #ul안에 있는 li를 list에다가 넣어줌  [<li></li>....]
print(lis)
lis[0].text

lis_col = []
lis_col.extend(lis[2])
lis_col.extend(lis[3])
lis_col.extend(lis[4])
lis_col.extend(lis[6])
#내가 필요한 열의 종류는 지역(사업명/위치), 규모(면적), 가격, 연도(분양/입주시기)) 
print(lis_col)
lis_col[0] = '지역명'
lis_col[1] = '규모구분'
lis_col[3] = '연도'
print(lis_col)

lis_col_frame = pd.DataFrame(lis_col)
lis_col_frame
#lis_col을 표로 나타낼 열의 정보로 가공하였다.

#----------------------------------------------------------------------

Citycell = {}

#지역에 대한 리스트
w_city = soup.find_all('td', {'class':'subject'})
#다 서울이기 때문에 모든 리스트의 요소를 '서울'로 간단하게 바꿔줌. 

for i in range(0, 15):   #0~14
    w_city[i]
    print(w_city[i].text)
print('\n')

for i in range(0, 15):
    w_city[i] = "서울"
    
print(w_city)
print('\n')

#연도에 대한 리스트 필요.
w_year = []

Citycell['지역명'] = w_city

sd = pd.DataFrame(Citycell)
sd = sd[['지역명']]

#----------------------------------------------------------------------
w_size = soup.find_all('b')
for i in range(0, 14):
    del w_size[i]

w_size[0] = "전용면적 102㎡초과"
w_size[1] = "전용면적 60㎡이하"
w_size[2] = "전용면적 102㎡초과"
w_size[3] = "전용면적 60㎡초과 85㎡이하"
w_size[4] = "전용면적 85㎡초과 102㎡이하"
w_size[5] = "전용면적 102㎡초과"
w_size[6] = "전용면적 60㎡이하"
w_size[7] = "전용면적 102㎡초과"
w_size[8] = "전용면적 60㎡이하"
w_size[9] = "전용면적 60㎡이하"
w_size[10] = "전용면적 60㎡이하"
w_size[11] = "전용면적 102㎡초과"
w_size[12] = "전용면적 102㎡초과"
w_size[13] = "전용면적 102㎡초과"
w_size[14] = "전용면적 102㎡초과"
print(w_size)
Citycell['지역명'] = w_city
Citycell['규모구분'] = w_size
sd = pd.DataFrame(Citycell)

sd = sd[['지역명', '규모구분']]
sd

#----------------------------------------------------------------------
print('\n')
w_price = soup.find_all('p')
print(w_price)               #6번째 주기로 가격정보가 나오는걸 알 수 있다.
                            #'미정'처리된 두 지역은 빼고 생각하였다.
print('\n')

w_price_l = []
for i in range(1,16):
    print(w_price[6*i-1].text)   
    w_price_l.extend(w_price[6*i-1])   #등차수열 이용.
print('\n')
print(w_price_l)
    
Citycell['지역명'] = w_city
Citycell['규모구분'] = w_size
Citycell['분양가격'] = w_price_l
sd = pd.DataFrame(Citycell)

sd = sd[['지역명', '규모구분','분양가격']]

#----------------------------------------------------------------------

#크롤링한 사이트는 단위가 '만 원'이다.
#본래 데이터분석을 한 자료의 단위는 '천 원'이므로 끝에 0을 더 붙여서 할당하였다.
w_price_l[0] = 350290
w_price_l[1] = 681580
w_price_l[2] = 828000
w_price_l[3] = 356800
w_price_l[4] = 289800
w_price_l[5] = 497100
w_price_l[6] = 399000
w_price_l[7] = 1058280
w_price_l[8] = 2500000
w_price_l[9] = 731100
w_price_l[10] = 648500
w_price_l[11] = 723300
w_price_l[12] = 149900
w_price_l[13] = 585200
w_price_l[14] = 552800
print('\n')
print(w_price_l)

sd = pd.DataFrame(Citycell)
sd = sd[['지역명', '규모구분','분양가격']]


#----------------------------------------------------------------------

#평당분양가격
sd['분양가격'] = pd.to_numeric(w_price_l, errors = 'coerce')

w_square = []

for i in range(0, 15):
    w_square.append(w_price_l[i])
print(w_square)
w_square[0] = int(350290/60)
w_square[1] = int(681580/102)
w_square[2] = int(828000/102)
w_square[3] = int(356800/60)
w_square[4] = int(289800/60)
w_square[5] = int(497100/102)
w_square[6] = int(399000/85)
w_square[7] = int(1058280/102)
w_square[8] = int(2500000/85)
w_square[9] = int(731100/102)
w_square[10] = int(648500/102)
w_square[11] = int(723300/102)
w_square[12] = int(149900/60)
w_square[13] = int(585200/102)
w_square[14] = int(552800/102)


Citycell['평당분양가격'] = w_square
sd = pd.DataFrame(Citycell)
sd = sd[['지역명', '규모구분','분양가격','평당분양가격']]


#----------------------------------------------------------------------

#연도에 대한 리스트 필요.
w_year = soup.find_all('p')
print(w_year)
#가격과 연도는 같은 <p>태그 안에 있다.
#분양가를 알아보는 프로젝트 이므로 입주일이 아닌 분양일 기준으로 리스트를 정리할 계획이다.
print('\n')
#7, 13, 19, 25 a+(n-1)d
#7+(i-1)*6        // 등차수열 이용.
w_year_l = []
for i in range(1,16):
    print(w_year[7+(i-1)*6].text)    
    w_year_l.extend(w_year[7+(i-1)*6])
    
    
    #extend(!@#$%^.text) 할때와 extend(#$%^&*()) 할때 차이가 나는이유???
print('\n')

print(w_year_l)
print('\n')

print(w_year_l)
w_year_l[0] = '2018'
for i in range(1,15):
    w_year_l[i] = '2017'
print('\n')
print(w_year_l)
Citycell['지역명'] = w_city
Citycell['규모구분'] = w_size
Citycell['가격'] = w_price_l
Citycell['연도'] = w_year_l

sd = pd.DataFrame(Citycell)
sd = sd[['지역명', '규모구분','분양가격','평당분양가격', '연도']]
sd

#----------------------------------------------------------------------


#----------------------------------------------------------------------


#----------------------------------------------------------------------


#----------------------------------------------------------------------



<ul class="top">
<li style="width:10%;">사업단계</li>
<li style="width:12%;">분양종류/형태</li>
<li style="width:30%;">사업명/위치</li>
<li style="width:12%;">면적</li>
<li style="width:10%;">가격</li>
<li style="width:12%;">세대수</li>
<li style="width:14%;">분양/입주시기</li>
</ul>
[<li style="width:10%;">사업단계</li>, <li style="width:12%;">분양종류/형태</li>, <li style="width:30%;">사업명/위치</li>, <li style="width:12%;">면적</li>, <li style="width:10%;">가격</li>, <li style="width:12%;">세대수</li>, <li style="width:14%;">분양/입주시기</li>]
['사업명/위치', '면적', '가격', '분양/입주시기']
['지역명', '규모구분', '가격', '연도']
사보이시티잠실 서울시 송파구 방이동 42-2,4,5 번지
e편한세상강동에코포레 서울특별시 강동구 길동 43
힐스테이트클래시안 서울특별시 영등포구 신길동 240-16
잠실벨솔레 서울특별시 송파구 방이동 41-8
가산센트럴푸르지오시티 서울특별시 금천구 가산동 233-5
서울항동제일풍경채(7BL) 서울특별시 구로구 항동 7BL
북한산두산위브2차 서울특별시 서대문구 홍은 1동 13-25번
고덕아르테온 서울특별시 강동구 상일동 121
문래롯데캐슬(뉴스테이) 서울특별시 영등포구 문래동6가 21
녹번역e편한세상캐슬 서울특별시 은평구 응암동 36, 37, 53번
백련산해모로 서울특별시 은평구 응암동 455-25
사가정센트럴아이파크 서울특별시 중랑구 면목동 164-10
칸타빌레5차 서울특별시 구로구 오류동 59-20
면목라온프라이빗 서울특별시 중랑구 면목동 171-7
휘경해모로프레스티지 서울

,지역명,규모구분,분양가격,평당분양가격,연도
0,서울,전용면적 102㎡초과,350290,5838,2018
1,서울,전용면적 60㎡이하,681580,6682,2017
2,서울,전용면적 102㎡초과,828000,8117,2017
3,서울,전용면적 60㎡초과 85㎡이하,356800,5946,2017
4,서울,전용면적 85㎡초과 102㎡이하,289800,4830,2017
5,서울,전용면적 102㎡초과,497100,4873,2017
6,서울,전용면적 60㎡이하,399000,4694,2017
7,서울,전용면적 102㎡초과,1058280,10375,2017
8,서울,전용면적 60㎡이하,2500000,29411,2017
9,서울,전용면적 60㎡이하,731100,7167,2017
